In [1]:
using PyCall
import PoseComposition: Pose, IDENTITY_POSE, IDENTITY_ORN
import Rotations as R

function __init__()
    global o3d = PyCall.pyimport("open3d");
    PyCall.py"""
    import open3d as o3d
    import numpy as np
    """
end
__init__()

In [3]:
function pose_to_transformation_matrix(pose::Pose)::Matrix
    transform = zeros(4,4)
    transform[1:3,1:3] .= Matrix(R.RotMatrix{3}(pose.orientation))
    transform[1:3, 4] .= pose.pos
    transform[4,4] = 1.
    transform
end

function make_axes(size::Real=1.0)
    return o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
end

function move_mesh_to_pose(m, pose::Pose)
    m.transform(pose_to_transformation_matrix(pose))
    m
end

function add(geometry; update=true)
    vis.add_geometry(geometry)
    if update 
        sync() 
    end
end

function sync()
    vis.poll_events()
    vis.update_renderer()
end

sync (generic function with 1 method)

### get data to visualize

In [ ]:
import JLD
import PoseComposition: Pose, IDENTITY_POSE, IDENTITY_ORN
import Rotations as R
import StaticArrays

In [28]:
data = JLD.load("pf_traces.jld")

Dict{String, Any} with 4 entries:
  "gt_pose_ori" => [-0.350653, 0.184601, -0.249942, 0.883456]
  "trace_info"  => Tuple{Float64, StaticArrays.SVector{4, Float64}}[(-3293.3, […
  "gt_pose_pos" => [0.0, 0.0, 5.0]
  "gt_obj_id"   => 1

In [31]:
# process JLD

gt_obj_id = data["gt_obj_id"]
@show gt_pose_ori = R.QuatRotation(data["gt_pose_ori"])
@show gt_pose = Pose(pos=data["gt_pose_pos"], orientation=gt_pose_ori)
traces = [(trace[1], R.QuatRotation(trace[2])) for trace in data["trace_info"]]


gt_pose_ori = R.QuatRotation(data["gt_pose_ori"]) = [-0.6859300732558756 0.5272925866063544 0.5014603301490256; -0.7118515197246732 -0.6291430970229324 -0.3121640231257706; 0.1508885299446667 -0.5710879893487288 0.8069021997446556]
gt_pose = Pose(pos = data["gt_pose_pos"], orientation = gt_pose_ori) = Pose⟨pos=[0.0, 0.0, 5.0], orientation=(w=-0.35065261636905254, x=0.1846014788824844, y=-0.24994238160438492, z=0.8834556256574894)⟩


15-element Vector{Tuple{Float64, QuatRotation{Float64}}}:
 (-3293.298472949294, [-0.6927053805835386 0.5233721255856485 0.4962266355916166; -0.7083795815724727 -0.6229856229474158 -0.33179403552810993; 0.13549031006615653 -0.5813523301413528 0.8022916203709198])
 (-3293.298472949294, [-0.6927053805835386 0.5233721255856485 0.4962266355916166; -0.7083795815724727 -0.6229856229474158 -0.33179403552810993; 0.13549031006615653 -0.5813523301413528 0.8022916203709198])
 (-3293.298472949294, [-0.6927053805835386 0.5233721255856485 0.4962266355916166; -0.7083795815724727 -0.6229856229474158 -0.33179403552810993; 0.13549031006615653 -0.5813523301413528 0.8022916203709198])
 (-3293.298472949294, [-0.6927053805835386 0.5233721255856485 0.4962266355916166; -0.7083795815724727 -0.6229856229474158 -0.33179403552810993; 0.13549031006615653 -0.5813523301413528 0.8022916203709198])
 (-3293.298472949294, [-0.6927053805835386 0.5233721255856485 0.4962266355916166; -0.7083795815724727 -0.6229856229474158 

### Visualize with axes
Size = (normalized) probability of particle 

In [4]:
vis = o3d.visualization.Visualizer()
vis.create_window()

true

In [5]:
a1 = make_axes(1.0)
a1 = move_mesh_to_pose(a1, IDENTITY_POSE)
add(a1)

In [6]:
a2 = make_axes(0.5)
a2 = move_mesh_to_pose(a2, Pose(ones(3), R.RotXYZ(0.1, 0.4, -2.0)))
add(a2)

In [7]:
a3 = make_axes(0.1)
a3 = move_mesh_to_pose(a3, Pose(-2.0 * ones(3), R.RotXYZ(-1.1, 0.4, -1.0)))
add(a3)